In [1]:
!pip install plotly==5.6.0

     |████████████████████████████████| 27.7 MB 19.9 MB/s eta 0:00:01


In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)

import os
from io import StringIO
import requests as r
from requests.auth import HTTPBasicAuth
from datetime import datetime

In [3]:
###########
# Constants
###########

JENKINS_URL = 'http://jenkins:8080'
JENKINS_USER = os.getenv('JENKINS_USER')
JENKINS_PASS = os.getenv('JENKINS_PASS')
JENKINS_JOB_NAME = 'kubedepend'

In [4]:
custom_date_parser = lambda x: datetime.strptime(x, "%m-%d-%Y_%H-%M-%S.%f")

In [5]:
#########
# Get results from Jenkins and organize them into one DataFrame
#########
session = r.Session()
session.auth = (JENKINS_USER, JENKINS_PASS)

res = session.get(f'{JENKINS_URL}/job/{JENKINS_JOB_NAME}/api/json')
data = res.json()
builds = data['builds']

results_raw = pd.DataFrame()
build_parameters = pd.DataFrame()

for build_summary in builds:
    build_number = build_summary['number']
    res = session.get(f'{JENKINS_URL}/job/{JENKINS_JOB_NAME}/{build_number}/api/json')
    build = res.json()
    artifact_list = build['artifacts']
    if build['result'] == 'SUCCESS' and artifact_list:
        for artifact in artifact_list:
            # Get builds, where there is a 'results.csv' artifact
            if artifact['fileName'] == 'results.csv':
                # get results
                res = session.get(f'{JENKINS_URL}/job/{JENKINS_JOB_NAME}/{build_number}/artifact/{artifact["relativePath"]}')
                meas_results_df = pd.read_csv(StringIO(res.text), delimiter=',', parse_dates=['measurement_seq_start_time', 'measurement_start_time', 'measurement_end_time'], date_parser=custom_date_parser)
                results_raw = results_raw.append(meas_results_df, ignore_index=True)
                
                # meas_results_df['id'] is a Series
                meas_seq_id = meas_results_df['id'][0]
            
                # extract parameter info
                parameterItems = list(filter(lambda a: a and a['_class'] == 'hudson.model.ParametersAction' ,build['actions']))[0]['parameters']
                parameters = list(filter(lambda p: p['_class'] in ['hudson.model.BooleanParameterValue', 'hudson.model.StringParameterValue'], parameterItems))
                reshaped = dict()
                for parameter in parameters:
                    reshaped[parameter['name']] = parameter['value']
                # add build number
                reshaped['build_number'] = build_number
                reshaped['id'] = meas_seq_id
                build_parameters = build_parameters.append(reshaped, ignore_index=True)
                

In [6]:
# 'results_raw' always stores all the data from Jenkins
# if an error occurs, the 'results' can be reset without querying Jenkins again
results = results_raw

In [7]:
# Clean build_parameters
build_parameters = build_parameters.fillna(False)
build_parameters['CleanEnhancements'] = build_parameters['CleanEnhancements'].astype(bool)
build_parameters['DeleteCFStack'] = build_parameters['DeleteCFStack'].astype(bool)
build_parameters['CleanWorkSpace'] = build_parameters['CleanWorkSpace'].astype(bool)
build_parameters['DeployTools'] = build_parameters['DeployTools'].astype(bool)
build_parameters['UseKafka'] = build_parameters['UseKafka'].astype(bool)
build_parameters['UseHeartbeats'] = build_parameters['UseHeartbeats'].astype(bool)
build_parameters['build_number'] = build_parameters['build_number'].astype(int)

build_parameters.insert(0, 'build_number', build_parameters.pop('build_number'))
build_parameters.insert(0, 'id', build_parameters.pop('id'))


build_parameters.head()
# build_parameters.info()

,id,build_number,CleanEnhancements,DeleteCFStack,CleanWorkSpace,CFStackName,EKSClusterName,AWSRegion,S3Bucket,DeployTools,...,RunMeasurement,MEASFaultProfile,MEASMinMeasurementCount,MEASMaxMeasurementCount,MEASTargetStd,MEASLoadDuration,MEASClusterType,MEASLocustUserCount,MEASLocustSpawnRate,MEASComment
0,11e90a50-30fa-495c-ae3b-8fec9890c9f1,446,False,False,True,morova-eks-cluster,morova-eks-cluster,us-east-2,morova-bucket,False,...,1.0,pod-kill,5,15,0.1,300,eks,1,1,final
1,077b7acd-17a9-4424-ad6f-122a1ee44a01,445,False,False,True,morova-eks-cluster,morova-eks-cluster,us-east-2,morova-bucket,False,...,True,pod-failure,5,15,0.1,300,eks,1,1,final
2,642fc9ca-a246-4a2c-b602-b73ab3079f26,442,False,False,True,morova-eks-cluster,morova-eks-cluster,us-east-2,morova-bucket,False,...,True,pod-kill,5,15,0.1,300,eks,1,1,final
3,449d367f-18a9-4183-8167-93a654b219e0,434,False,False,True,morova-eks-cluster,morova-eks-cluster,us-east-2,morova-bucket,False,...,True,pod-kill,5,15,0.1,300,eks,1,1,final
4,44e0909a-7766-485e-aad6-381a73027237,433,False,False,True,morova-eks-cluster,morova-eks-cluster,us-east-2,morova-bucket,False,...,True,pod-kill,5,15,0.1,300,eks,1,1,final


In [8]:
# Select relevant columns
build_parameters = build_parameters.loc[:,['id', 'build_number', 'UseKafka', 'UseHeartbeats']].reset_index(drop=True)
build_parameters = build_parameters.rename(columns={'UseKafka': 'use_kafka', 'UseHeartbeats': 'use_heartbeats'})
build_parameters.head()

,id,build_number,use_kafka,use_heartbeats
0,11e90a50-30fa-495c-ae3b-8fec9890c9f1,446,True,True
1,077b7acd-17a9-4424-ad6f-122a1ee44a01,445,True,True
2,642fc9ca-a246-4a2c-b602-b73ab3079f26,442,True,False
3,449d367f-18a9-4183-8167-93a654b219e0,434,False,True
4,44e0909a-7766-485e-aad6-381a73027237,433,False,True


In [9]:
# for development, get only base measurements
results = results[results['comment'] == 'final']
# results.head()
results

,id,measurement_seq_start_time,availability,mut,mdt,mtbf,measurement_start_time,measurement_end_time,submitted_jobs,finished_jobs,fault_profile,cluster_type,measurement_count,load_duration,locust_user_count,locust_spawn_rate,prev_stack_git_commit_short,comment
0,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,1.000000,1.000000,0.000000,1.000000,2022-04-04 12:10:28.290622,2022-04-04 12:15:37.361417,59.0,54.0,pod-kill,eks,1,300,1,1,9e98e83,final
1,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,1.000000,1.000000,0.000000,1.000000,2022-04-04 12:17:02.965367,2022-04-04 12:22:11.708618,NaN,NaN,pod-kill,eks,2,300,1,1,9e98e83,final
2,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,0.950000,0.475000,0.050000,0.525000,2022-04-04 12:23:01.574798,2022-04-04 12:28:10.829707,117.0,83.0,pod-kill,eks,3,300,1,1,9e98e83,final
3,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,0.950000,0.475000,0.050000,0.525000,2022-04-04 12:30:41.029417,2022-04-04 12:35:49.050787,58.0,44.0,pod-kill,eks,4,300,1,1,9e98e83,final
4,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,0.950000,0.475000,0.050000,0.525000,2022-04-04 12:42:08.594717,2022-04-04 12:47:16.902474,60.0,40.0,pod-kill,eks,5,300,1,1,9e98e83,final
5,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.750000,0.375000,0.250000,0.625000,2022-04-04 09:00:42.370809,2022-04-04 09:05:51.811130,46.0,27.0,pod-failure,eks,1,300,1,1,9e98e83,final
6,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.550000,0.275000,0.450000,0.725000,2022-04-04 09:07:48.918626,2022-04-04 09:12:57.904616,35.0,17.0,pod-failure,eks,2,300,1,1,9e98e83,final
7,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.300000,0.300000,0.700000,1.000000,2022-04-04 09:14:21.788679,2022-04-04 09:19:29.510015,20.0,7.0,pod-failure,eks,3,300,1,1,9e98e83,final
8,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.200000,0.200000,0.800000,1.000000,2022-04-04 09:20:53.947971,2022-04-04 09:26:02.792524,12.0,4.0,pod-failure,eks,4,300,1,1,9e98e83,final
9,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.050000,0.050000,0.475000,0.525000,2022-04-04 09:26:21.756630,2022-04-04 09:31:30.064328,8.0,4.0,pod-failure,eks,5,300,1,1,9e98e83,final


In [10]:
# Set meas count to the maximum of the measuremenet count in the same measurement sequence

# select columns
results_meas_count = results.loc[:, ['id', 'measurement_count']]
# group by max value
results_meas_max_count = results_meas_count.groupby(by=['id']).max()
# rename
results_meas_max_count = results_meas_max_count.rename(columns={'measurement_count': 'measurement_count_max'})
# merge with results
results = results.merge(results_meas_max_count, on='id').drop(columns=['measurement_count']).rename(columns={'measurement_count_max': 'measurement_count'})
results

,id,measurement_seq_start_time,availability,mut,mdt,mtbf,measurement_start_time,measurement_end_time,submitted_jobs,finished_jobs,fault_profile,cluster_type,load_duration,locust_user_count,locust_spawn_rate,prev_stack_git_commit_short,comment,measurement_count
0,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,1.000000,1.000000,0.000000,1.000000,2022-04-04 12:10:28.290622,2022-04-04 12:15:37.361417,59.0,54.0,pod-kill,eks,300,1,1,9e98e83,final,5
1,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,1.000000,1.000000,0.000000,1.000000,2022-04-04 12:17:02.965367,2022-04-04 12:22:11.708618,NaN,NaN,pod-kill,eks,300,1,1,9e98e83,final,5
2,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,0.950000,0.475000,0.050000,0.525000,2022-04-04 12:23:01.574798,2022-04-04 12:28:10.829707,117.0,83.0,pod-kill,eks,300,1,1,9e98e83,final,5
3,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,0.950000,0.475000,0.050000,0.525000,2022-04-04 12:30:41.029417,2022-04-04 12:35:49.050787,58.0,44.0,pod-kill,eks,300,1,1,9e98e83,final,5
4,11e90a50-30fa-495c-ae3b-8fec9890c9f1,2022-04-04 12:10:18.006784,0.950000,0.475000,0.050000,0.525000,2022-04-04 12:42:08.594717,2022-04-04 12:47:16.902474,60.0,40.0,pod-kill,eks,300,1,1,9e98e83,final,5
5,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.750000,0.375000,0.250000,0.625000,2022-04-04 09:00:42.370809,2022-04-04 09:05:51.811130,46.0,27.0,pod-failure,eks,300,1,1,9e98e83,final,15
6,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.550000,0.275000,0.450000,0.725000,2022-04-04 09:07:48.918626,2022-04-04 09:12:57.904616,35.0,17.0,pod-failure,eks,300,1,1,9e98e83,final,15
7,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.300000,0.300000,0.700000,1.000000,2022-04-04 09:14:21.788679,2022-04-04 09:19:29.510015,20.0,7.0,pod-failure,eks,300,1,1,9e98e83,final,15
8,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.200000,0.200000,0.800000,1.000000,2022-04-04 09:20:53.947971,2022-04-04 09:26:02.792524,12.0,4.0,pod-failure,eks,300,1,1,9e98e83,final,15
9,077b7acd-17a9-4424-ad6f-122a1ee44a01,2022-04-04 09:00:31.822339,0.050000,0.050000,0.475000,0.525000,2022-04-04 09:26:21.756630,2022-04-04 09:31:30.064328,8.0,4.0,pod-failure,eks,300,1,1,9e98e83,final,15


In [11]:
#  group by measurement ID and get mean of numerical properties
results_avg_by_id = results.groupby(by=['id']).mean()
# results_avg_by_id.head()
results_avg_by_id

,availability,mut,mdt,mtbf,submitted_jobs,finished_jobs,load_duration,locust_user_count,locust_spawn_rate,measurement_count
id,,,,,,,,,,
03631a77-d080-44bf-a582-718e4c39084d,0.325439,0.229630,NaN,NaN,44.777778,44.444444,300.0,1.0,1.0,9.0
077b7acd-17a9-4424-ad6f-122a1ee44a01,0.426667,0.255000,0.384444,0.639444,33.466667,16.200000,300.0,1.0,1.0,15.0
0f52ec58-1ae2-482d-9012-6c642d16bade,0.920000,0.531667,0.050000,0.581667,72.500000,65.500000,300.0,1.0,1.0,5.0
11e90a50-30fa-495c-ae3b-8fec9890c9f1,0.970000,0.685000,0.030000,0.715000,73.500000,55.250000,300.0,1.0,1.0,5.0
28c950b7-e744-46aa-80fa-fc429e62d912,0.524912,0.283333,0.284444,0.567778,45.833333,32.500000,300.0,1.0,1.0,15.0
2912fcdb-b8a0-4c23-87a9-632fb1fa7762,0.920000,0.531667,0.050000,0.581667,58.250000,55.000000,300.0,1.0,1.0,5.0
2b2484d5-302d-4448-ad74-f4413b11a1ba,0.950000,0.575000,0.040000,0.615000,70.500000,66.250000,300.0,1.0,1.0,5.0
387bd9f2-511a-4516-9d78-c58d533ac364,0.736316,0.395556,0.151111,0.546667,55.200000,54.600000,300.0,1.0,1.0,15.0
42e912b3-5cc4-47bc-a518-672fd0c0d01f,0.980000,0.980000,0.020000,1.000000,60.000000,60.000000,300.0,1.0,1.0,5.0


In [12]:
# get non-numerical data for each measurement sequence
results_non_numerical = results.loc[:, ['id', 'fault_profile', 'measurement_seq_start_time', 'cluster_type', 'comment']] \
    .drop_duplicates() \
    .reset_index(drop=True)
# results_non_numerical.head()
results_non_numerical

,id,fault_profile,measurement_seq_start_time,cluster_type,comment
0,11e90a50-30fa-495c-ae3b-8fec9890c9f1,pod-kill,2022-04-04 12:10:18.006784,eks,final
1,077b7acd-17a9-4424-ad6f-122a1ee44a01,pod-failure,2022-04-04 09:00:31.822339,eks,final
2,642fc9ca-a246-4a2c-b602-b73ab3079f26,pod-kill,2022-04-03 19:50:36.678766,eks,final
3,449d367f-18a9-4183-8167-93a654b219e0,pod-kill,2022-04-03 13:50:00.617719,eks,final
4,44e0909a-7766-485e-aad6-381a73027237,pod-kill,2022-04-03 13:15:23.869493,eks,final
5,e4b79268-bbe4-44a7-8c1d-e640b9d4e634,pod-kill,2022-04-03 12:32:18.120779,eks,final
6,a325098c-965d-4dbf-8c73-33296ac77d03,pod-failure,2022-04-03 10:51:11.781131,eks,final
7,e63d8cbb-0f7b-487e-8339-4fc0a1839ef5,pod-kill,2022-04-03 09:53:51.786775,eks,final
8,851328f9-dc75-4d2c-a4fe-3ebd54e8e9ac,pod-kill,2022-04-03 09:08:01.368185,eks,final
9,28c950b7-e744-46aa-80fa-fc429e62d912,pod-failure,2022-04-02 18:34:17.993822,eks,final


In [13]:
# merge the grouped by avg results with non-numerical data
results = pd.merge(results_avg_by_id, results_non_numerical, on='id')

# merge the results with build parameters
results = pd.merge(results, build_parameters, on='id')

# filter out measurement sequence with less than MIN_MEASUREMENT_COUNT measurements
# results = results[results['measurement_count'] >= MIN_MEASUREMENT_COUNT]

# results.head()
results

,id,availability,mut,mdt,mtbf,submitted_jobs,finished_jobs,load_duration,locust_user_count,locust_spawn_rate,measurement_count,fault_profile,measurement_seq_start_time,cluster_type,comment,build_number,use_kafka,use_heartbeats
0,077b7acd-17a9-4424-ad6f-122a1ee44a01,0.426667,0.255000,0.384444,0.639444,33.466667,16.200000,300.0,1.0,1.0,15.0,pod-failure,2022-04-04 09:00:31.822339,eks,final,445,True,True
1,0f52ec58-1ae2-482d-9012-6c642d16bade,0.920000,0.531667,0.050000,0.581667,72.500000,65.500000,300.0,1.0,1.0,5.0,pod-kill,2022-03-31 07:55:35.485558,eks,final,408,False,False
2,11e90a50-30fa-495c-ae3b-8fec9890c9f1,0.970000,0.685000,0.030000,0.715000,73.500000,55.250000,300.0,1.0,1.0,5.0,pod-kill,2022-04-04 12:10:18.006784,eks,final,446,True,True
3,28c950b7-e744-46aa-80fa-fc429e62d912,0.524912,0.283333,0.284444,0.567778,45.833333,32.500000,300.0,1.0,1.0,15.0,pod-failure,2022-04-02 18:34:17.993822,eks,final,422,True,False
4,2912fcdb-b8a0-4c23-87a9-632fb1fa7762,0.920000,0.531667,0.050000,0.581667,58.250000,55.000000,300.0,1.0,1.0,5.0,pod-kill,2022-03-29 18:42:57.866807,eks,final,392,False,False
5,2b2484d5-302d-4448-ad74-f4413b11a1ba,0.950000,0.575000,0.040000,0.615000,70.500000,66.250000,300.0,1.0,1.0,5.0,pod-kill,2022-03-30 08:47:38.627685,eks,final,396,False,False
6,387bd9f2-511a-4516-9d78-c58d533ac364,0.736316,0.395556,0.151111,0.546667,55.200000,54.600000,300.0,1.0,1.0,15.0,network-delay,2022-03-30 16:12:44.120109,eks,final,403,False,False
7,42e912b3-5cc4-47bc-a518-672fd0c0d01f,0.980000,0.980000,0.020000,1.000000,60.000000,60.000000,300.0,1.0,1.0,5.0,none,2022-03-29 12:24:31.624449,eks,final,387,False,False
8,449d367f-18a9-4183-8167-93a654b219e0,0.970000,0.685000,0.030000,0.715000,77.000000,75.666667,300.0,1.0,1.0,5.0,pod-kill,2022-04-03 13:50:00.617719,eks,final,434,False,True
9,44e0909a-7766-485e-aad6-381a73027237,0.959474,0.570000,0.050000,0.620000,96.000000,92.333333,300.0,1.0,1.0,5.0,pod-kill,2022-04-03 13:15:23.869493,eks,final,433,False,True


In [14]:
# drop rows where there are NaN data
results = results.dropna()

# drop rows where mtbf metrics have > 1 value
results = results[results['mtbf'] <= 1]

# drop rows where comment is 'test'
resutls = results[results['comment'] != 'test']

# group by fault profile and only keep the newest measurement
time_column = 'measurement_seq_start_time'
latest_idx = results.groupby(['fault_profile', 'use_kafka', 'use_heartbeats'])[time_column].transform('max') == results[time_column]

results = results[latest_idx]
results

,id,availability,mut,mdt,mtbf,submitted_jobs,finished_jobs,load_duration,locust_user_count,locust_spawn_rate,measurement_count,fault_profile,measurement_seq_start_time,cluster_type,comment,build_number,use_kafka,use_heartbeats
0,077b7acd-17a9-4424-ad6f-122a1ee44a01,0.426667,0.255000,0.384444,0.639444,33.466667,16.200000,300.0,1.0,1.0,15.0,pod-failure,2022-04-04 09:00:31.822339,eks,final,445,True,True
2,11e90a50-30fa-495c-ae3b-8fec9890c9f1,0.970000,0.685000,0.030000,0.715000,73.500000,55.250000,300.0,1.0,1.0,5.0,pod-kill,2022-04-04 12:10:18.006784,eks,final,446,True,True
3,28c950b7-e744-46aa-80fa-fc429e62d912,0.524912,0.283333,0.284444,0.567778,45.833333,32.500000,300.0,1.0,1.0,15.0,pod-failure,2022-04-02 18:34:17.993822,eks,final,422,True,False
6,387bd9f2-511a-4516-9d78-c58d533ac364,0.736316,0.395556,0.151111,0.546667,55.200000,54.600000,300.0,1.0,1.0,15.0,network-delay,2022-03-30 16:12:44.120109,eks,final,403,False,False
8,449d367f-18a9-4183-8167-93a654b219e0,0.970000,0.685000,0.030000,0.715000,77.000000,75.666667,300.0,1.0,1.0,5.0,pod-kill,2022-04-03 13:50:00.617719,eks,final,434,False,True
11,642fc9ca-a246-4a2c-b602-b73ab3079f26,0.835714,0.344048,0.094048,0.438095,93.000000,78.250000,300.0,1.0,1.0,7.0,pod-kill,2022-04-03 19:50:36.678766,eks,final,442,True,False
15,8ab0e4f9-6224-4a65-97b1-196b787b0dc4,1.000000,1.000000,0.000000,1.000000,60.000000,60.000000,300.0,1.0,1.0,5.0,none,2022-03-29 15:05:56.856168,eks,final,390,False,False
16,903f2465-335e-4991-8ab6-8d0bd8446406,1.000000,1.000000,0.000000,1.000000,59.400000,59.400000,300.0,1.0,1.0,5.0,stress-cpu,2022-03-31 11:03:54.694877,eks,final,410,False,False
18,a325098c-965d-4dbf-8c73-33296ac77d03,0.598947,0.335000,0.279444,0.614444,36.800000,33.533333,300.0,1.0,1.0,15.0,pod-failure,2022-04-03 10:51:11.781131,eks,final,431,False,True
21,cd3a38d1-4acd-4594-9492-303f32e60b29,0.544912,0.208889,0.201111,0.410000,37.666667,37.666667,300.0,1.0,1.0,5.0,io,2022-03-23 20:12:07.239328,eks,final,357,False,False


In [15]:
def enhancement_labeling(row):
    if (row['use_kafka'] and row['use_heartbeats']):
        return 'Kafka_And_Heartbeats'
    if (row['use_kafka']):
        return 'Kafka'
    if (row['use_heartbeats']):
        return 'Heartbeats'
    return 'None'

In [16]:
results['enhancement'] = results.apply(enhancement_labeling, axis=1)
results = results.drop(columns=['use_kafka', 'use_heartbeats'])
results

,id,availability,mut,mdt,mtbf,submitted_jobs,finished_jobs,load_duration,locust_user_count,locust_spawn_rate,measurement_count,fault_profile,measurement_seq_start_time,cluster_type,comment,build_number,enhancement
0,077b7acd-17a9-4424-ad6f-122a1ee44a01,0.426667,0.255000,0.384444,0.639444,33.466667,16.200000,300.0,1.0,1.0,15.0,pod-failure,2022-04-04 09:00:31.822339,eks,final,445,Kafka_And_Heartbeats
2,11e90a50-30fa-495c-ae3b-8fec9890c9f1,0.970000,0.685000,0.030000,0.715000,73.500000,55.250000,300.0,1.0,1.0,5.0,pod-kill,2022-04-04 12:10:18.006784,eks,final,446,Kafka_And_Heartbeats
3,28c950b7-e744-46aa-80fa-fc429e62d912,0.524912,0.283333,0.284444,0.567778,45.833333,32.500000,300.0,1.0,1.0,15.0,pod-failure,2022-04-02 18:34:17.993822,eks,final,422,Kafka
6,387bd9f2-511a-4516-9d78-c58d533ac364,0.736316,0.395556,0.151111,0.546667,55.200000,54.600000,300.0,1.0,1.0,15.0,network-delay,2022-03-30 16:12:44.120109,eks,final,403,None
8,449d367f-18a9-4183-8167-93a654b219e0,0.970000,0.685000,0.030000,0.715000,77.000000,75.666667,300.0,1.0,1.0,5.0,pod-kill,2022-04-03 13:50:00.617719,eks,final,434,Heartbeats
11,642fc9ca-a246-4a2c-b602-b73ab3079f26,0.835714,0.344048,0.094048,0.438095,93.000000,78.250000,300.0,1.0,1.0,7.0,pod-kill,2022-04-03 19:50:36.678766,eks,final,442,Kafka
15,8ab0e4f9-6224-4a65-97b1-196b787b0dc4,1.000000,1.000000,0.000000,1.000000,60.000000,60.000000,300.0,1.0,1.0,5.0,none,2022-03-29 15:05:56.856168,eks,final,390,None
16,903f2465-335e-4991-8ab6-8d0bd8446406,1.000000,1.000000,0.000000,1.000000,59.400000,59.400000,300.0,1.0,1.0,5.0,stress-cpu,2022-03-31 11:03:54.694877,eks,final,410,None
18,a325098c-965d-4dbf-8c73-33296ac77d03,0.598947,0.335000,0.279444,0.614444,36.800000,33.533333,300.0,1.0,1.0,15.0,pod-failure,2022-04-03 10:51:11.781131,eks,final,431,Heartbeats
21,cd3a38d1-4acd-4594-9492-303f32e60b29,0.544912,0.208889,0.201111,0.410000,37.666667,37.666667,300.0,1.0,1.0,5.0,io,2022-03-23 20:12:07.239328,eks,final,357,None


In [17]:
# Save results
results.to_csv('data/results_cleaned.csv')